In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-CSE-CIC-IDS2018-v3.parquet"
data = pd.read_parquet(file_name)

In [3]:
data.Label.value_counts()

Label
0    17514626
1     2600903
Name: count, dtype: int64

In [4]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [5]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [6]:
data.Attack.unique()

array(['Benign', 'FTP-BruteForce', 'SSH-Bruteforce',
       'DoS_attacks-GoldenEye', 'DoS_attacks-Slowloris',
       'DoS_attacks-SlowHTTPTest', 'DoS_attacks-Hulk',
       'DDoS_attacks-LOIC-HTTP', 'DDOS_attack-LOIC-UDP',
       'DDOS_attack-HOIC', 'Brute_Force_-Web', 'Brute_Force_-XSS',
       'SQL_Injection', 'Infilteration', 'Bot'], dtype=object)

In [7]:
data = data.groupby(by='Attack').sample(frac=0.02, random_state=13)

In [8]:
data.groupby(by="Attack").count()

,FLOW_START_MILLISECONDS,FLOW_END_MILLISECONDS,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,FTP_COMMAND_RET_CODE,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,350293,350293,350293,350293,350293,350293,350293,350293,350293,350293,...,350293,350293,350293,350293,350293,350293,350293,350293,350293,350293
Bot,4154,4154,4154,4154,4154,4154,4154,4154,4154,4154,...,4154,4154,4154,4154,4154,4154,4154,4154,4154,4154
Brute_Force_-Web,32,32,32,32,32,32,32,32,32,32,...,32,32,32,32,32,32,32,32,32,32
Brute_Force_-XSS,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
DDOS_attack-HOIC,20646,20646,20646,20646,20646,20646,20646,20646,20646,20646,...,20646,20646,20646,20646,20646,20646,20646,20646,20646,20646
DDOS_attack-LOIC-UDP,69,69,69,69,69,69,69,69,69,69,...,69,69,69,69,69,69,69,69,69,69
DDoS_attacks-LOIC-HTTP,5772,5772,5772,5772,5772,5772,5772,5772,5772,5772,...,5772,5772,5772,5772,5772,5772,5772,5772,5772,5772
DoS_attacks-GoldenEye,1226,1226,1226,1226,1226,1226,1226,1226,1226,1226,...,1226,1226,1226,1226,1226,1226,1226,1226,1226,1226
DoS_attacks-Hulk,2002,2002,2002,2002,2002,2002,2002,2002,2002,2002,...,2002,2002,2002,2002,2002,2002,2002,2002,2002,2002


In [9]:
# X = data.drop(columns=["Attack", "Label"])
X = data.drop(columns=["Attack", "Label", "FLOW_START_MILLISECONDS", "FLOW_END_MILLISECONDS",
                       "SRC_TO_DST_IAT_MIN", "SRC_TO_DST_IAT_MAX", "SRC_TO_DST_IAT_AVG",
                       "SRC_TO_DST_IAT_STDDEV", "DST_TO_SRC_IAT_MIN", "DST_TO_SRC_IAT_MAX",
                       "DST_TO_SRC_IAT_AVG", "DST_TO_SRC_IAT_STDDEV"])
y = data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)

In [10]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [11]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [12]:
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [13]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h
18744204,172.31.69.14,18.219.211.138,4.740102e-07,1.323114e-06,0.001264,0.000012,0.000801,0.000012,1.045428e-06,1.016509e-06,...,0.0,0.019249,0.063169,3.165538e-07,3.165538e-07,4.626324e-07,4.626324e-07,0.000000,3.038169e-07,"[4.7401022514464884e-07, 1.3231141133088038e-0..."
16092777,172.31.64.33,169.254.169.254,8.691857e-08,2.426175e-07,0.000155,0.000002,0.000196,0.000002,1.177649e-07,1.037055e-07,...,0.0,0.003530,0.007722,5.804601e-08,5.804601e-08,8.483223e-08,8.483223e-08,0.000000,5.571047e-08,"[8.691857374655046e-08, 2.4261753340371224e-07..."
8585764,18.219.9.1,172.31.69.28,3.946205e-07,1.101512e-06,0.001047,0.000010,0.002244,0.000010,8.703343e-07,8.462588e-07,...,0.0,0.128201,0.052589,2.635357e-07,2.635357e-07,3.851483e-07,3.851483e-07,0.000000,2.529321e-07,"[3.946205413164243e-07, 1.1015121191910754e-06..."
9151507,18.218.229.235,172.31.69.28,2.759324e-07,7.702155e-07,0.000756,0.000007,0.001569,0.000007,6.085679e-07,5.917335e-07,...,0.0,0.089643,0.036772,1.842733e-07,1.842733e-07,2.693090e-07,2.693090e-07,0.000000,1.768589e-07,"[2.759323680481234e-07, 7.702154745117165e-07,..."
14212479,172.31.65.38,172.31.0.2,8.558578e-07,8.375243e-07,0.003386,0.000059,0.010988,0.000059,8.503230e-07,8.503230e-07,...,0.0,0.000000,0.000000,8.001776e-06,8.001776e-06,6.389747e-06,8.525785e-07,0.003564,7.679815e-06,"[8.558577987023313e-07, 8.375242601934381e-07,..."


In [14]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [15]:
# Training graph
train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [16]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [17]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [18]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [19]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [20]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [21]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [22]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [23]:
# Convert to GPU
train_g = train_g.to('cuda')

In [24]:
# cnt_wait = 0
# best = 1e9
# best_t = 0
# dur = []
# node_features = train_g.ndata['h'] 
# edge_features = train_g.edata['h']
# torch.cuda.empty_cache()

# for epoch in range(epochs):
#     dgi.train()
#     if epoch >= 3:
#         t0 = time.time()

#     dgi_optimizer.zero_grad()
#     loss = dgi(train_g, node_features, edge_features)
#     loss.backward()
#     dgi_optimizer.step()

#     if loss < best:
#         best = loss
#         best_t = epoch
#         cnt_wait = 0
#         torch.save(dgi.state_dict(), 'best_dgi_CSE_v3.pkl')
#     else:
#         cnt_wait += 1

#   # if cnt_wait == patience:
#   #     print('Early stopping!')
#   #     break

#     if epoch >= 3:
#         dur.append(time.time() - t0)

#     if epoch % 50 == 0:

#         print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
#             "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
#               loss.item(),
#               train_g.num_edges() / np.mean(dur) / 1000))

In [25]:
dgi.load_state_dict(torch.load('best_dgi_CSE_v3.pkl'))

/tmp/ipykernel_12132/204607001.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dgi.load_state_dict(torch.load('best_dgi_CSE_v3.pkl'))


<All keys matched successfully>

In [26]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [27]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [28]:
# Convert to GPU
test_g = test_g.to('cuda')

In [29]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [30]:
df_train = pd.DataFrame(training_emb, )
df_train["Attack"] = lab_enc.inverse_transform(
        train_g.edata['Attack'].detach().cpu().numpy())
df_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb, )
df_test["Attack"] = lab_enc.inverse_transform(
        test_g.edata['Attack'].detach().cpu().numpy())
df_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [31]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,0.004144,0.025403,0.040047,-0.000309,0.030387,0.035868,0.138171,0.057873,0.010817,-0.054248,...,0.005187,-0.014806,0.054438,-0.032151,0.051813,0.010525,-0.104780,-0.039206,Bot,1
1,0.004144,0.025403,0.040047,-0.000309,0.030387,0.035868,0.138171,0.057873,0.010817,-0.054248,...,0.005187,-0.014806,0.054438,-0.032151,0.051813,0.010525,-0.104780,-0.039206,Bot,1
2,0.004144,0.025403,0.040047,-0.000309,0.030387,0.035868,0.138171,0.057873,0.010817,-0.054248,...,0.005187,-0.014806,0.054438,-0.032151,0.051813,0.010525,-0.104780,-0.039206,Bot,1
3,0.004144,0.025403,0.040047,-0.000309,0.030387,0.035868,0.138171,0.057873,0.010817,-0.054248,...,0.005187,-0.014806,0.054438,-0.032151,0.051813,0.010525,-0.104780,-0.039206,Bot,1
4,0.004144,0.025403,0.040047,-0.000309,0.030387,0.035868,0.138171,0.057873,0.010817,-0.054248,...,0.005187,-0.014806,0.054438,-0.032151,0.051813,0.010525,-0.104780,-0.039206,Bot,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561750,0.019681,0.034812,0.099503,-0.011409,0.065458,0.058795,0.224296,0.088324,0.012971,-0.026900,...,-0.031089,-0.015791,0.062819,-0.076096,0.074027,-0.094270,-0.179397,-0.084104,Benign,0
561751,0.012349,0.043348,0.095392,-0.051661,0.044234,0.075227,0.223614,0.103547,-0.000789,-0.043036,...,-0.039768,-0.038003,0.078018,-0.083092,0.080884,-0.071523,-0.171710,-0.075815,Benign,0
561752,0.001127,0.030448,0.066351,-0.033875,0.056376,0.050331,0.160193,0.084940,-0.004454,-0.040618,...,-0.021869,-0.030398,0.059316,-0.068465,0.082230,-0.026403,-0.106622,-0.047486,Benign,0
561753,0.021613,0.035513,0.086050,0.003022,0.070035,0.067304,0.194402,0.073408,0.019399,-0.023465,...,-0.001308,-0.026009,0.032723,-0.087231,0.073443,-0.059594,-0.145814,-0.047920,Benign,0


# Embeddings CBLOF  Embeddings

In [32]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [33]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [34]:
test_samples

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,-0.010756,0.017720,0.024812,-0.002006,0.040568,0.003775,0.123052,0.086528,-0.027222,-0.021224,...,-0.108371,0.019642,0.018567,-0.013365,0.018211,-0.069348,0.079747,0.020226,-0.046336,-0.028960
1,-0.003213,0.020006,0.024880,-0.005277,0.030732,0.013256,0.110244,0.066249,-0.011410,-0.030661,...,-0.096846,-0.000706,0.021885,-0.021149,0.018321,-0.058501,0.059623,0.013894,-0.067156,-0.016921
2,0.002488,0.029440,0.085328,-0.004804,0.048028,0.030766,0.204019,0.097525,-0.003975,-0.049451,...,-0.119103,0.018997,-0.020523,-0.023260,0.027365,-0.055817,0.059882,-0.024887,-0.160084,-0.086254
3,0.002488,0.029440,0.085328,-0.004804,0.048028,0.030766,0.204019,0.097525,-0.003975,-0.049451,...,-0.119103,0.018997,-0.020523,-0.023260,0.027365,-0.055817,0.059882,-0.024887,-0.160084,-0.086254
4,0.012453,0.003061,0.036308,0.011524,0.039038,0.041112,0.118182,0.047699,0.019957,-0.049117,...,-0.069926,0.003132,-0.003112,0.015622,0.063140,-0.028199,0.065774,0.020224,-0.097738,-0.019758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240747,0.041802,0.050003,0.033959,0.017936,0.028507,0.048082,0.182786,0.041863,0.029330,-0.020081,...,-0.096479,0.013980,0.027313,0.004784,0.030921,-0.092017,0.079428,-0.021408,-0.108641,-0.044897
240748,0.004536,0.032466,0.072623,-0.035304,0.054304,0.053864,0.172574,0.087451,-0.003309,-0.040056,...,-0.138155,-0.028234,-0.026702,-0.034199,0.059426,-0.071682,0.079313,-0.036924,-0.120119,-0.053163
240749,0.031656,0.025944,0.055078,0.035175,0.053992,0.046049,0.153323,0.054340,0.028498,-0.018457,...,-0.088087,-0.000265,0.015934,-0.003539,0.023146,-0.066473,0.068461,-0.031990,-0.095027,-0.040611
240750,0.006054,0.020481,0.105378,-0.016414,0.082273,0.061779,0.192417,0.093325,0.014141,-0.025521,...,-0.152730,-0.000619,-0.028335,-0.040189,0.036486,-0.081170,0.068944,-0.085648,-0.153584,-0.049767


In [35]:
df_raw_train = pd.concat([X_train.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_train], axis=1)
df_raw_test = pd.concat([X_test.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_test], axis=1)

In [36]:
df_raw_train

,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Attack,Label
18744204,4.740102e-07,1.323114e-06,0.001264,1.174887e-05,0.000801,1.174887e-05,1.045428e-06,1.016509e-06,1.189494e-06,2.584752e-05,...,0.019249,0.063169,3.165538e-07,3.165538e-07,4.626324e-07,4.626324e-07,0.000000,3.038169e-07,Bot,1
16092777,8.691857e-08,2.426175e-07,0.000155,2.154374e-06,0.000196,2.154374e-06,1.177649e-07,1.037055e-07,2.181158e-07,4.308748e-07,...,0.003530,0.007722,5.804601e-08,5.804601e-08,8.483223e-08,8.483223e-08,0.000000,5.571047e-08,Benign,0
8585764,3.946205e-07,1.101512e-06,0.001047,9.781112e-06,0.002244,9.781112e-06,8.703343e-07,8.462588e-07,9.902714e-07,3.716823e-05,...,0.128201,0.052589,2.635357e-07,2.635357e-07,3.851483e-07,3.851483e-07,0.000000,2.529321e-07,DDOS_attack-HOIC,1
9151507,2.759324e-07,7.702155e-07,0.000756,6.839293e-06,0.001569,6.839293e-06,6.085679e-07,5.917335e-07,6.924321e-07,1.778216e-05,...,0.089643,0.036772,1.842733e-07,1.842733e-07,2.693090e-07,2.693090e-07,0.000000,1.768589e-07,DDOS_attack-HOIC,1
14212479,8.558578e-07,8.375243e-07,0.003386,5.939708e-05,0.010988,5.939708e-05,8.503230e-07,8.503230e-07,8.198402e-07,5.405134e-03,...,0.000000,0.000000,8.001776e-06,8.001776e-06,6.389747e-06,8.525785e-07,0.003564,7.679815e-06,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7181547,5.916211e-07,5.789478e-07,0.002792,4.105888e-05,0.006282,4.105888e-05,5.877951e-07,5.877951e-07,5.667235e-07,2.217180e-03,...,0.000000,0.000000,5.531315e-06,5.531315e-06,4.340304e-06,5.319259e-07,0.002464,5.308757e-06,Benign,0
3012536,6.281540e-07,6.806339e-09,0.000125,3.113900e-06,0.000000,0.000000e+00,1.724675e-08,1.261226e-06,4.298023e-08,3.113900e-06,...,0.084209,0.000000,4.194942e-07,4.194942e-07,6.130762e-07,6.130762e-07,0.000000,4.026154e-07,Benign,0
345028,1.170458e-08,1.145385e-08,0.000058,8.123050e-07,0.000110,8.123050e-07,1.162888e-08,1.162888e-08,1.121200e-08,8.123050e-07,...,0.000000,0.000000,1.094310e-07,1.094310e-07,8.880476e-08,1.165973e-08,0.000048,1.050279e-07,Benign,0
19662289,1.473345e-05,1.410984e-07,0.036372,3.651849e-04,0.027900,3.651849e-04,1.362926e-06,1.927213e-06,8.815569e-08,5.112589e-02,...,0.598319,0.598319,9.839298e-06,9.839298e-06,1.437979e-05,1.437979e-05,0.000000,9.443403e-06,Benign,0


In [37]:
raw_benign_train_samples = df_raw_train[df_raw_train.Label == 0].drop(columns=["Label", "Attack"])
raw_normal_train_samples = df_raw_train.drop(columns=["Label", "Attack"])

raw_train_labels = df_raw_train["Label"]
raw_test_labels = df_raw_test["Label"]

raw_test_samples = df_raw_test.drop(columns=["Label", "Attack"])

In [38]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [39]:
from pyod.models.cblof import CBLOF
n_est = [2,3,5,7,9,10]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    try:
        clf_if.fit(benign_train_samples)
    except Exception as e:
        continue
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [02:25<00:00,  4.04s/it]

{'n_estimators': 5, 'con': 0.04}
0.4853113069317739
              precision    recall  f1-score   support

           0     0.8700    0.9576    0.9117    209540
           1     0.1205    0.0390    0.0590     31212

    accuracy                         0.8385    240752
   macro avg     0.4952    0.4983    0.4853    240752
weighted avg     0.7728    0.8385    0.8011    240752



In [40]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    try:
        clf_if.fit(normal_train_samples)
    except Exception as e:
        continue
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [02:51<00:00,  4.75s/it]

{'n_estimators': 7, 'con': 0.04}
0.4854076119559081
              precision    recall  f1-score   support

           0     0.8699    0.9563    0.9110    209540
           1     0.1195    0.0399    0.0598     31212

    accuracy                         0.8375    240752
   macro avg     0.4947    0.4981    0.4854    240752
weighted avg     0.7726    0.8375    0.8007    240752



In [41]:
###  CBLOF RAW

In [42]:
from pyod.models.cblof import CBLOF

n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = CBLOF(n_clusters=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

  3%|▎         | 1/36 [00:00<00:08,  3.89it/s]

2


  6%|▌         | 2/36 [00:00<00:08,  3.85it/s]

2


...

3


 33%|███▎      | 12/36 [00:03<00:06,  3.50it/s]

3


100%|██████████| 36/36 [00:22<00:00,  1.58it/s]

{'n_estimators': 7, 'con': 0.1}
0.7033441573276542
              precision    recall  f1-score   support

           0     0.9313    0.8966    0.9137    105088
           1     0.4436    0.5549    0.4930     15606

    accuracy                         0.8525    120694
   macro avg     0.6875    0.7257    0.7033    120694
weighted avg     0.8683    0.8525    0.8593    120694



In [43]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  3%|▎         | 1/36 [00:00<00:09,  3.71it/s]

2


  6%|▌         | 2/36 [00:00<00:08,  3.79it/s]

2


  8%|▊         | 3/36 [00:00<00:09,  3.64it/s]

...

3


 36%|███▌      | 13/36 [00:03<00:06,  3.49it/s]

5


100%|██████████| 36/36 [00:21<00:00,  1.69it/s]

benign only
{'n_estimators': 7}
0.6717299555628837
              precision    recall  f1-score   support

           0     0.9322    0.8528    0.8908    105088
           1     0.3702    0.5825    0.4527     15606

    accuracy                         0.8179    120694
   macro avg     0.6512    0.7177    0.6717    120694
weighted avg     0.8596    0.8179    0.8341    120694



In [44]:
# HBOS  Embeddings

In [45]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [46]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [04:04<00:00,  6.79s/it]

{'n_estimators': 20, 'con': 0.05}
0.46919392889007994
              precision    recall  f1-score   support

           0     0.8668    0.9466    0.9049    209540
           1     0.0606    0.0231    0.0335     31212

    accuracy                         0.8269    240752
   macro avg     0.4637    0.4849    0.4692    240752
weighted avg     0.7622    0.8269    0.7919    240752



In [47]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [04:27<00:00,  7.44s/it]

{'n_estimators': 10, 'con': 0.05}
0.4667707508539166
              precision    recall  f1-score   support

           0     0.8659    0.9395    0.9012    209540
           1     0.0539    0.0231    0.0324     31212

    accuracy                         0.8207    240752
   macro avg     0.4599    0.4813    0.4668    240752
weighted avg     0.7606    0.8207    0.7885    240752



In [48]:
##  HBOS  RAw

In [49]:
from pyod.models.cblof import CBLOF

n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = HBOS(n_bins=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:20<00:00,  1.75it/s]

{'n_estimators': 5, 'con': 0.2}
0.5891832154718102
              precision    recall  f1-score   support

           0     0.9083    0.8023    0.8520    105088
           1     0.2545    0.4545    0.3263     15606

    accuracy                         0.7574    120694
   macro avg     0.5814    0.6284    0.5892    120694
weighted avg     0.8238    0.7574    0.7841    120694



In [50]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = HBOS(n_bins=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:21<00:00,  1.70it/s]

benign only
{'n_estimators': 5}
0.5661071163679476
              precision    recall  f1-score   support

           0     0.8858    0.9172    0.9012    105088
           1     0.2673    0.2034    0.2310     15606

    accuracy                         0.8249    120694
   macro avg     0.5765    0.5603    0.5661    120694
weighted avg     0.8058    0.8249    0.8146    120694



In [51]:
##  PCA  Emb

In [52]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [01:42<00:00,  2.84s/it]

{'n_estimators': 20, 'con': 0.05}
0.46826737716285166
              precision    recall  f1-score   support

           0     0.8666    0.9466    0.9049    209540
           1     0.0575    0.0219    0.0317     31212

    accuracy                         0.8268    240752
   macro avg     0.4621    0.4843    0.4683    240752
weighted avg     0.7617    0.8268    0.7917    240752



In [53]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [03:03<00:00,  5.10s/it]

{'n_estimators': 15, 'con': 0.05}
0.46694483452928304
              precision    recall  f1-score   support

           0     0.8662    0.9430    0.9030    209540
           1     0.0537    0.0217    0.0309     31212

    accuracy                         0.8236    240752
   macro avg     0.4599    0.4824    0.4669    240752
weighted avg     0.7608    0.8236    0.7899    240752



In [54]:
##  PCA  RAw

In [55]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples)
   
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:29<00:00,  1.21it/s]

{'n_estimators': 5, 'con': 0.1}
0.5080176311636
              precision    recall  f1-score   support

           0     0.8727    0.8997    0.8860    105088
           1     0.1471    0.1165    0.1300     15606

    accuracy                         0.7984    120694
   macro avg     0.5099    0.5081    0.5080    120694
weighted avg     0.7789    0.7984    0.7883    120694



In [56]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples)

    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:30<00:00,  1.17it/s]

benign only
{'n_estimators': 10}
0.4991731366592155
              precision    recall  f1-score   support

           0     0.8707    0.9000    0.8851    105088
           1     0.1298    0.1004    0.1132     15606

    accuracy                         0.7966    120694
   macro avg     0.5003    0.5002    0.4992    120694
weighted avg     0.7749    0.7966    0.7853    120694



In [57]:
##  IF  Emb

In [58]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [02:30<00:00,  6.26s/it]

{'n_estimators': 50, 'con': 0.04}
0.4860949035247128
              precision    recall  f1-score   support

           0     0.8700    0.9557    0.9108    209540
           1     0.1213    0.0411    0.0614     31212

    accuracy                         0.8371    240752
   macro avg     0.4956    0.4984    0.4861    240752
weighted avg     0.7729    0.8371    0.8007    240752



In [59]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [02:44<00:00,  6.85s/it]

{'n_estimators': 150, 'con': 0.05}
0.48078568451828124
              precision    recall  f1-score   support

           0     0.8684    0.9428    0.9041    209540
           1     0.0964    0.0409    0.0575     31212

    accuracy                         0.8259    240752
   macro avg     0.4824    0.4919    0.4808    240752
weighted avg     0.7683    0.8259    0.7943    240752



In [60]:
##  IF  Raw

In [61]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples.to_numpy())
   
    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:42<00:00,  1.76s/it]

{'n_estimators': 100, 'con': 0.1}
0.5774755972050177
              precision    recall  f1-score   support

           0     0.8900    0.8992    0.8946    105088
           1     0.2702    0.2512    0.2604     15606

    accuracy                         0.8154    120694
   macro avg     0.5801    0.5752    0.5775    120694
weighted avg     0.8098    0.8154    0.8126    120694



In [62]:
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples.to_numpy())

    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:45<00:00,  1.89s/it]

benign only
{'n_estimators': 20}
0.534376213973836
              precision    recall  f1-score   support

           0     0.8841    0.8121    0.8466    105088
           1     0.1828    0.2832    0.2222     15606

    accuracy                         0.7437    120694
   macro avg     0.5335    0.5476    0.5344    120694
weighted avg     0.7934    0.7437    0.7658    120694

